## Homework: Deep Jazz

In [1]:
import numpy as np
from music21 import stream, midi, tempo, note

#import lstm
from grammar import unparse_grammar
from preprocess import get_musical_data, get_corpus_data

from qa import prune_grammar, prune_notes, clean_up_notes
from generator import __sample, __generate_grammar, __predict

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
N_epochs = 128  # default
data_fn = 'midi/' + 'original_metheny.mid'  # 'And Then I Knew' by Pat Metheny
out_fn = 'midi/' 'deepjazz_on_metheny...' + str(N_epochs)

In [3]:
max_len = 20
max_tries = 1000
diversity = 0.5

# musical settings
bpm = 130

# get data
chords, abstract_grammars = get_musical_data(data_fn)
corpus, values, val_indices, indices_val = get_corpus_data(abstract_grammars)
print('corpus length:', len(corpus))
print('total # of values:', len(values))

corpus length: 193
total # of values: 78


In [4]:
chords[0]

[<music21.instrument.Piano Piano>,
 <music21.tempo.MetronomeMark Quarter=112.0>,
 <music21.key.Key of G major>,
 <music21.meter.TimeSignature 4/4>]

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

def get_keras_model(max_len, N_values):
    # build a 2 stacked LSTM
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(max_len, N_values)))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(N_values))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Task

Replace previous model with equivalent in prettytensor or tf.slim

Try to make you code as compact as possible

In [6]:
import tflearn
tflearn.init_graph(gpu_memory_fraction=0.33)

gpu_options {
  per_process_gpu_memory_fraction: 0.33
}
allow_soft_placement: true

In [7]:
def get_tflearn_model(max_len, N_values):   
    net = tflearn.input_data(shape=[None, max_len, N_values])
    net = tflearn.lstm(net, 128, dropout=(1.0, 0.8), return_seq=True)
    net = tflearn.lstm(net, 128, dropout=(1.0, 0.8), return_seq=False)
    net = tflearn.fully_connected(net, N_values, activation='softmax')
    net = tflearn.regression(net, optimizer='rmsprop', loss='categorical_crossentropy')
    model = tflearn.DNN(net)
    model.rec_predict = model.predict
    model.predict = lambda x, **kwargs: model.rec_predict(x)
    return model

In [8]:
get_model = get_tflearn_model

In [9]:
import numpy as np

''' Build a 2-layer LSTM from a training corpus '''


def build_model(corpus, val_indices, max_len, N_epochs=128):
    # number of different values or words in corpus
    N_values = len(set(corpus))

    # cut the corpus into semi-redundant sequences of max_len values
    step = 3
    sentences = []
    next_values = []
    for i in range(0, len(corpus) - max_len, step):
        sentences.append(corpus[i: i + max_len])
        next_values.append(corpus[i + max_len])
    print('nb sequences:', len(sentences))

    # transform data into binary matrices
    X = np.zeros((len(sentences), max_len, N_values), dtype=np.bool)
    y = np.zeros((len(sentences), N_values), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, val in enumerate(sentence):
            X[i, t, val_indices[val]] = 1
        y[i, val_indices[next_values[i]]] = 1

    model = get_model(max_len, N_values)
    model.fit(X, y, batch_size=128, n_epoch=N_epochs) #epochs=N_epochs)

    return model


In [10]:
# build model
model = build_model(corpus=corpus, val_indices=val_indices,
                         max_len=max_len, N_epochs=N_epochs)

nb sequences: 58
---------------------------------
Run id: KIL9DN
Log directory: /tmp/tflearn_logs/
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
---------------------------------
Training samples: 58
Validation samples: 0
--
Training Step: 1  | time: 0.612s
| RMSProp | epoch: 001 | loss: 0.00000 -- iter: 58/58
--
Training Step: 2  | total loss: 3.92220 | time: 0.068s
| RMSProp | epoch: 002 | loss: 3.92220 -- iter: 58/58
--
Training Step: 3  | total loss: 4.27855 | time: 0.068s
| RMSProp | epoch: 003 | loss: 4.27855 -- iter: 58/58
--
Training Step: 4  | total loss: 4.33780 | time: 0.070s
| RMSProp | epoch: 004 | loss: 4.33780 -- iter: 58/58
--
Training Step: 5  | total loss: 4.35157 | time: 0.076s
| RMSProp | epoch: 005 | loss: 4.35157 -- iter: 58/58
--
Training Step: 6  | total loss: 4.35510 | time: 0.075s
| RMSProp | epoch: 006 | loss: 4.35510 -- iter: 58/58
--
Training Step: 7  | total loss: 4.35691 | ti

Training Step: 62  | total loss: 4.34947 | time: 0.071s
| RMSProp | epoch: 062 | loss: 4.34947 -- iter: 58/58
--
Training Step: 63  | total loss: 4.34913 | time: 0.071s
| RMSProp | epoch: 063 | loss: 4.34913 -- iter: 58/58
--
Training Step: 64  | total loss: 4.34867 | time: 0.077s
| RMSProp | epoch: 064 | loss: 4.34867 -- iter: 58/58
--
Training Step: 65  | total loss: 4.34831 | time: 0.072s
| RMSProp | epoch: 065 | loss: 4.34831 -- iter: 58/58
--
Training Step: 66  | total loss: 4.34782 | time: 0.070s
| RMSProp | epoch: 066 | loss: 4.34782 -- iter: 58/58
--
Training Step: 67  | total loss: 4.34743 | time: 0.067s
| RMSProp | epoch: 067 | loss: 4.34743 -- iter: 58/58
--
Training Step: 68  | total loss: 4.34701 | time: 0.066s
| RMSProp | epoch: 068 | loss: 4.34701 -- iter: 58/58
--
Training Step: 69  | total loss: 4.34656 | time: 0.071s
| RMSProp | epoch: 069 | loss: 4.34656 -- iter: 58/58
--
Training Step: 70  | total loss: 4.34614 | time: 0.068s
| RMSProp | epoch: 070 | loss: 4.34614 -

Training Step: 125  | total loss: 4.03845 | time: 0.076s
| RMSProp | epoch: 125 | loss: 4.03845 -- iter: 58/58
--
Training Step: 126  | total loss: 4.00288 | time: 0.068s
| RMSProp | epoch: 126 | loss: 4.00288 -- iter: 58/58
--
Training Step: 127  | total loss: 3.96809 | time: 0.073s
| RMSProp | epoch: 127 | loss: 3.96809 -- iter: 58/58
--
Training Step: 128  | total loss: 3.93386 | time: 0.076s
| RMSProp | epoch: 128 | loss: 3.93386 -- iter: 58/58
--


In [12]:
# set up audio stream
out_stream = stream.Stream()

# generation loop
curr_offset = 0.0
loopEnd = len(chords)
for loopIndex in range(1, loopEnd):
    # get chords from file
    curr_chords = stream.Voice()
    for j in chords[loopIndex]:
        curr_chords.insert((j.offset % 4), j)

    # generate grammar
    curr_grammar = __generate_grammar(model=model, corpus=corpus,
                                      abstract_grammars=abstract_grammars,
                                      values=values, val_indices=val_indices,
                                      indices_val=indices_val,
                                      max_len=max_len, max_tries=max_tries,
                                      diversity=diversity)

    curr_grammar = curr_grammar.replace(' A', ' C').replace(' X', ' C')

    # Pruning #1: smoothing measure
    curr_grammar = prune_grammar(curr_grammar)

    # Get notes from grammar and chords
    curr_notes = unparse_grammar(curr_grammar, curr_chords)

    # Pruning #2: removing repeated and too close together notes
    curr_notes = prune_notes(curr_notes)

    # quality assurance: clean up notes
    curr_notes = clean_up_notes(curr_notes)

    # print # of notes in curr_notes
    print('After pruning: %s notes' % (len([i for i in curr_notes
                                            if isinstance(i, note.Note)])))

    # insert into the output stream
    for m in curr_notes:
        out_stream.insert(curr_offset + m.offset, m)
    for mc in curr_chords:
        out_stream.insert(curr_offset + mc.offset, mc)

    curr_offset += 4.0

out_stream.insert(0.0, tempo.MetronomeMark(number=bpm))

# Play the final stream through output (see 'play' lambda function above)
play = lambda x: midi.realtime.StreamPlayer(x).play()
play(out_stream)

# save stream
mf = midi.translate.streamToMidiFile(out_stream)
mf.open(out_fn, 'wb')
mf.write()
mf.close()    

After pruning: 13 notes
After pruning: 13 notes
After pruning: 12 notes
After pruning: 15 notes
After pruning: 13 notes
After pruning: 14 notes
After pruning: 13 notes
After pruning: 13 notes
After pruning: 14 notes
After pruning: 14 notes
After pruning: 12 notes
After pruning: 14 notes
After pruning: 12 notes
After pruning: 15 notes
After pruning: 14 notes
After pruning: 13 notes
After pruning: 13 notes
After pruning: 14 notes


You can play generated sample using any midi player

Under linux I prefer timidity

In [18]:
!! timidity midi/deepjazz_on_metheny...128_epochs.midi

['Requested buffer size 32768, fragment size 8192',
 "ALSA pcm 'default' set buffer size 32768, period size 8192 bytes",
 'Playing midi/deepjazz_on_metheny...128_epochs.midi',
 'MIDI file: midi/deepjazz_on_metheny...128_epochs.midi',
 'midi/deepjazz_on_metheny...128_epochs.midi: No such file or directory']